### 1. KOSIS 국가통계포털 데이터 불러오기

In [38]:
import pandas as pd
import requests
from pandas.io.json import json_normalize

# 데이터 접속 주소 입력
url = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=ZjU4ZmIzNGIyYjIzZWRmOTY0NGZlZWEwOTUxYTE1YTM=&format=json&jsonVD=Y&userStatsId=dlgudtjq333/101/DT_1G1B002/2/1/20190418170806&prdSe=M&newEstPrdCnt=1'
# 정상 여부 확인 (200 정상)
response = requests.get(url)
response

# JSON 데이터 획득
json = response.json()

# PandasDataframe 변환
df = json_normalize(json)
df.head()

df.to_csv('./kosisData1.csv', encoding = "ms949")

### 2. 공공데이터 포털  API 데이터 불러오기 (XML)

In [1]:
import pandas as pd
import urllib.request

#### ① url받기 xml타입

In [4]:
xmlurl = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D&pageNo=1&startPage=1&numOfRows=10&pageSize=10&LAWD_CD=11110&DEAL_YMD=201512"

#### ② xml 데이터 불러오기

In [10]:
request = urllib.request.Request(xmlurl) 
request.get_method = lambda: 'GET' 
response_body = urllib.request.urlopen(request).read() 
fullXmlString = str(response_body, "utf-8")

In [11]:
fullXmlString

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>    82,500</거래금액><건축년도>2008</건축년도><년>2015</년><도로명>사직로8길</도로명><도로명건물본번호코드>00004</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>11110</도로명시군구코드><도로명일련번호코드>03</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4100135</도로명코드><법정동> 사직동</법정동><법정동본번코드>0009</법정동본번코드><법정동부번코드>0000</법정동부번코드><법정동시군구코드>11110</법정동시군구코드><법정동읍면동코드>11500</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>광화문풍림스페이스본(101동~105동)</아파트><월>12</월><일>1~10</일><일련번호>11110-2203</일련번호><전용면적>94.51</전용면적><지번>9</지번><지역코드>11110</지역코드><층>11</층></item><item><거래금액>    60,000</거래금액><건축년도>1981</건축년도><년>2015</년><도로명>세종대로23길</도로명><도로명건물본번호코드>00047</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>11110</도로명시군구코드><도로명일련번호코드>02</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4100190</도로명코드><법정동> 당주동</법정동><법정동본번코드>0145</법정동본번코드><법정동부번코드>0000</법정동부번코드><법정동시군구코드>11110</법정동시군구코드><법정동읍면동코드>11700</법정동

#### ③ xml 데이터 추출하기

In [12]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse, Element
# xml 데이터 추출할때 사용하는 라이브러리

In [14]:
# 모든 데이터를 담을 리스트 선언
result = []
# xml 라이브러리 표준
root = ET.fromstring(fullXmlString)
# body->items->item 모두 선택
elements = root.findall('body/items/item')

In [32]:
elements[1].find('거래금액').text

'    60,000'

In [34]:
# for 문을 돌면서 리스트에 모든 행 데이터 담기
result = []
root = ET.fromstring(fullXmlString)
elements = root.findall('body/items/item')
for item in elements:
    try:
        item_list = [] #거래금액, 건축년도 두개의 데이터만 담고 포문이 돌때마다 초기화
        item_list.append(item.find('거래금액').text)
        item_list.append(item.find('건축년도').text)
        result.append(item_list) #item_list를 초기화 하기전에 result에 item_list에 담겨있는 데이터를 담아줌.
    except Exception as e:
        print("This row will be ignored. ",item_list)
result

[['    82,500', '2008'],
 ['    60,000', '1981'],
 ['   130,000', '2004'],
 ['   105,000', '2004'],
 ['   120,000', '2003'],
 ['    17,000', '2014'],
 ['    17,000', '2014'],
 ['    57,000', '2006'],
 ['    44,000', '1995'],
 ['    52,000', '1995']]

#### ④ 추출데이터 데이터 프레임으로 변환하기

In [35]:
finalResult = pd.DataFrame(result)
finalResult.columns = ["거래금액","건축년도"]
finalResult.head()

,거래금액,건축년도
0,"82,500",2008
1,"60,000",1981
2,"130,000",2004
3,"105,000",2004
4,"120,000",2003
